In [1]:
#Importation des librairies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#Ne pas afficher les alertes
import warnings
warnings.simplefilter("ignore")

In [3]:
#Instruction permettant d'afficher toutes les colonnes d'un dataframe
pd.set_option('display.max_columns', None)

<div style="border: 1px solid RGB(40,165,165);" >
<h3 style="margin: auto; padding: 20px; color: RGB(40,165,165); ">Création de l'algorithme avec la Régression logistique</h3>
</div>

In [4]:
df_billets = pd.read_csv("billets.csv", sep=";")
df_billets_drop = df_billets.dropna(subset=["margin_low"])

In [5]:
# Je transforme les données True/False en 1 et 0 :
df_billets_drop_rlc = df_billets_drop.copy()
df_billets_drop_rlc['is_genuine'] = df_billets_drop_rlc['is_genuine'].replace({True: 1, False: 0})
df_billets_drop_rlc.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,1,171.81,104.86,104.95,4.52,2.89,112.83
1,1,171.46,103.36,103.66,3.77,2.99,113.09
2,1,172.69,104.48,103.50,4.40,2.94,113.16
3,1,171.36,103.91,103.94,3.62,3.01,113.51
4,1,171.73,104.28,103.46,4.04,3.48,112.54


In [6]:
# # Définition des caractéristiques (X) et de la cible (y)
X_rlc = df_billets_drop_rlc[["diagonal","height_left","height_right","margin_low","margin_up","length"]] # Toutes les colonnes sauf 'is_genuine'
y_rlc = df_billets_drop_rlc['is_genuine'] # La colonne 'is_genuine'
X_rlc_train, X_rlc_test, y_rlc_train, y_rlc_test = train_test_split(X_rlc, y_rlc, test_size=0.20, random_state=42)
# initialisation et entraînement de la Régression logistique classique
clf_rlc = LogisticRegression(random_state=42).fit(X_rlc_train, y_rlc_train)

In [7]:
# Fonction : 
def predict_billets_proba(model, input_dataframe):
    # Préparez les données
    dimensions_billet = input_dataframe[['diagonal','height_left','height_right','margin_low', 'margin_up','length']]

    # Utilisez le modèle de régression logistique pour faire des prédictions
    predicted_is_genuine = model.predict(dimensions_billet)
    predicted_proba = model.predict_proba(dimensions_billet)[:, 1]

    # Ajoutez les prédictions et les probabilités au DataFrame initial
    input_dataframe['predicted_is_genuine'] = predicted_is_genuine
    input_dataframe['predicted_proba'] = predicted_proba
    
    return input_dataframe

In [8]:
# Import du fichier à traiter
data = pd.read_csv("billets_test.csv")

In [9]:
# Test sur le fichier billets_production
# Utilisation la fonction de la Régression logistique classique avec le DataFrame à traiter
predicted_billets = predict_billets_proba(clf_rlc, data)

In [10]:
predicted_billets['predicted_is_genuine'] = predicted_billets['predicted_is_genuine'].replace({1: True, 0: False})
predicted_billets.head(30)

,diagonal,height_left,height_right,margin_low,margin_up,length,id,predicted_is_genuine,predicted_proba
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,True,0.987502
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,False,0.008237
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,True,0.998939
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,False,0.000151
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,False,0.012558


In [11]:
# Export en CSV 
predicted_billets.to_csv("result_prod_billets.csv", sep=';', index=False)